In [133]:
import numpy as np
from scipy.sparse import spdiags
import matplotlib.pyplot as plt

m = 64 # N value in x and y directions
n = m * m # total size of matrix
L = 20
dx = dy = L / m

e0 = np.zeros((n, 1)) # vector of zeros
e1 = np.ones((n, 1)) # vector of ones
e2 = np.copy(e1) # copy the one vector
e4 = np.copy(e0) # copy the zero vector
for j in range(1, m+1):
    e2[m*j-1] = 0 # overwrite every m^th value with zero
    e4[m*j-1] = 1 # overwirte every m^th value with one

# Shift to correct positions
e3 = np.zeros_like(e2)
e3[1:n] = e2[0:n-1]
e3[0] = e2[n-1]
e5 = np.zeros_like(e4)
e5[1:n] = e4[0:n-1]
e5[0] = e4[n-1]
# Place diagonal elements

diagonals = [e1.flatten(), e1.flatten(), e5.flatten(),
e2.flatten(), -4 * e1.flatten(), e3.flatten(),
e4.flatten(), e1.flatten(), e1.flatten()]
offset = [-(n-m), -m, -m+1, -1, 0, 1, m-1, m, (n-m)]
matA = spdiags(diagonals, offset, n, n).toarray() / (dx**2)
matA[0,0] = 2 / (dx**2)

A = matA

e1 = np.ones((n, 1))  
e2 = np.ones((n, 1))  
e3 = np.ones((n,1))  
e4 = np.ones((n, 1 ))  


diagonals_B = [e1.flatten(), -e2.flatten(), e3.flatten(), -e4.flatten()]
offsets_B = [-n+m, -m, m , n-m]  # Below and above neighbors (one step in y-direction)

matB = spdiags(diagonals_B, offsets_B, n, n).toarray() / ( 2 * dx )

B = matB

e1 = np.zeros((n, 1))  
e2 = np.ones((n, 1))  
e3 = np.copy(e2)  
e4 = np.copy(e1)  
for index in range(64):
    e1[64 * index]= 1
    e2[64 * index + 63] = 0
    e3[64 * index] = 0 
    e4[64 * index + 63]= 1


diagonals_C = [e1.flatten(), -e2.flatten(), e3.flatten(), -e4.flatten()]
offsets_C = [-m+1, -1, 1 , m-1]  # Below and above neighbors (one step in y-direction)

matC = spdiags(diagonals_C, offsets_C, n, n).toarray() / ( 2 * dy)


C = matC

import matplotlib.pyplot as plt
from scipy.fft import fft2, ifft2
from scipy.integrate import solve_ivp

# Parameters
nu = 0.001
Lx = 20
Ly = 20
nx = 64
ny = 64
N = nx * ny
t = np.arange(0, 4.5, 0.5)  # time array

# Create the spatial grid
x2 = np.linspace(-Lx/2, Lx/2, nx + 1)
x = x2[:nx]
y2 = np.linspace(-Ly/2, Ly/2, ny + 1)
y = y2[:ny]
X, Y = np.meshgrid(x, y)

# Initial vorticity (elliptical Gaussian)
w = np.exp( - X ** 2 - (1/20) * (Y**2))
w2 = w.reshape((N))

kx = (2 * np.pi / Lx) * np.concatenate((np.arange(0, nx/2), np.arange(-nx/2, 0)))
kx[0] = 1e-6
ky = (2 * np.pi / Ly) * np.concatenate((np.arange(0, ny/2), np.arange(-ny/2, 0)))
ky[0] = 1e-6
KX, KY = np.meshgrid(kx, ky)
K = KX**2 + KY**2

# Poisson equation solver (in Fourier space)

#def pde(nx, ny, K, N, B, C, A, nu, w2):
def pde(t, nu, C, B, A, ny, nx, K,w2 ):
    w = w2.reshape(( nx , ny ))
    wt = fft2(w)
    psit = - wt / K 
    psi = np.real(ifft2(psit)).reshape(nx * ny)
    psi_x = np.dot(B, psi)
    psi_y = np.dot(C, psi)
    wx = np.dot(B, w)
    wy = np.dot(C, w)
    rhs = nu * np.dot(A, w) - (wx * psi_y) + (psi_x * wy).reshape(nx * ny)
    return rhs

# Initial conditions (flattened initial vorticity)

#wsol = solve_ivp(lambda t, w_flat: pde(t, w2, nx, ny, A, B, C, nu), [t[0], t[-1]], w2, t_eval=t, method='RK45')
wsol = solve_ivp(pde, [t[0], t[-1]], w2, t_eval=t , args = (nx, ny, A, B, C, nu, K) , method = 'RK45')

A1 = wsol.y

TypeError: only integer scalar arrays can be converted to a scalar index

In [60]:
# (b) 
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import spdiags
from scipy.sparse.linalg import bicgstab, gmres
from scipy.linalg import lu
from scipy.integrate import solve_ivp
import time
from scipy.linalg import lu, solve_triangular

np.linalg.solve(A, w2)

P, L, U = lu(A)
Pb = np.dot(P, b)
y = solve_triangular(L, Pb, lower=True)
x = solve_triangular(U, y)


(4096,)

In [115]:
import numpy as np
from scipy.sparse import spdiags
import matplotlib.pyplot as plt

m = 64 # N value in x and y directions
n = m * m # total size of matrix
L = 20
dx = dy = L / m

e0 = np.zeros((n, 1)) # vector of zeros
e1 = np.ones((n, 1)) # vector of ones
e2 = np.copy(e1) # copy the one vector
e4 = np.copy(e0) # copy the zero vector
for j in range(1, m+1):
    e2[m*j-1] = 0 # overwrite every m^th value with zero
    e4[m*j-1] = 1 # overwirte every m^th value with one

# Shift to correct positions
e3 = np.zeros_like(e2)
e3[1:n] = e2[0:n-1]
e3[0] = e2[n-1]
e5 = np.zeros_like(e4)
e5[1:n] = e4[0:n-1]
e5[0] = e4[n-1]
# Place diagonal elements

diagonals = [e1.flatten(), e1.flatten(), e5.flatten(),
e2.flatten(), -4 * e1.flatten(), e3.flatten(),
e4.flatten(), e1.flatten(), e1.flatten()]
offset = [-(n-m), -m, -m+1, -1, 0, 1, m-1, m, (n-m)]
matA = spdiags(diagonals, offset, n, n).toarray() / (dx**2)
matA[0,0] = 2 / (dx**2)

A = matA

e1 = np.ones((n, 1))  
e2 = np.ones((n, 1))  
e3 = np.ones((n,1))  
e4 = np.ones((n, 1 ))  


diagonals_B = [e1.flatten(), -e2.flatten(), e3.flatten(), -e4.flatten()]
offsets_B = [-n+m, -m, m , n-m]  # Below and above neighbors (one step in y-direction)

matB = spdiags(diagonals_B, offsets_B, n, n).toarray() / ( 2 * dx )

B = matB

e1 = np.zeros((n, 1))  
e2 = np.ones((n, 1))  
e3 = np.copy(e2)  
e4 = np.copy(e1)  
for index in range(64):
    e1[64 * index]= 1
    e2[64 * index + 63] = 0
    e3[64 * index] = 0 
    e4[64 * index + 63]= 1


diagonals_C = [e1.flatten(), -e2.flatten(), e3.flatten(), -e4.flatten()]
offsets_C = [-m+1, -1, 1 , m-1]  # Below and above neighbors (one step in y-direction)

matC = spdiags(diagonals_C, offsets_C, n, n).toarray() / ( 2 * dy)


C = matC

import matplotlib.pyplot as plt
from scipy.fft import fft2, ifft2
from scipy.integrate import solve_ivp

# Parameters
nu = 0.001
Lx = 20
Ly = 20
nx = 64
ny = 64
N = nx * ny
t = np.arange(0, 4.5, 0.5)  # time array

# Create the spatial grid
x2 = np.linspace(-Lx/2, Lx/2, nx + 1)
x = x2[:nx]
y2 = np.linspace(-Ly/2, Ly/2, ny + 1)
y = y2[:ny]
X, Y = np.meshgrid(x, y)

# Initial vorticity (elliptical Gaussian)
w = np.exp( - X ** 2 - (1/20)*(Y**2))
w2 = w.reshape((N))

kx = (2 * np.pi / Lx) * np.concatenate((np.arange(0, nx/2), np.arange(-nx/2, 0)))
kx[0] = 1e-6
ky = (2 * np.pi / Ly) * np.concatenate((np.arange(0, ny/2), np.arange(-ny/2, 0)))
ky[0] = 1e-6
KX, KY = np.meshgrid(kx, ky)
K = KX**2 + KY**2

# Poisson equation solver (in Fourier space)

#def pde(nx, ny, K, N, B, C, A, nu, w2):
from scipy.fft import fft2, ifft2

def pde(t, w2, K, nx, ny, A, B, C, nu):
    # Reshape w2 back to the 2D grid
    w = w2.reshape((nx, ny))

    # Compute the Fourier transform of w
    wt = fft2(w)
    
    # Solve the Poisson equation in Fourier space
    psit = -wt / K
    psi = np.real(ifft2(psit)).reshape((nx * ny))
    # Compute the derivatives
    psi_x = np.dot(B, psi)
    psi_y = np.dot(C, psi)
    wx = np.dot(B, w.flatten())
    wy = np.dot(C, w.flatten())

    # Compute the right-hand side
    rhs = nu * np.dot(A, w.flatten()) - (wx * psi_y) + (psi_x * wy).reshape((nx * ny))

    return rhs

# Initial conditions (flattened initial vorticity)
w2 = w.flatten()
#wsol = solve_ivp(lambda t, w_flat: pde(t, w2, nx, ny, A, B, C, nu), [t[0], t[-1]], w2, t_eval=t, method='RK45')
wsol = solve_ivp(pde, [t[0], t[-1]], w2, t_eval=t , args = (nx, ny, A, B, C, nu, K) , method = 'RK45')

A1 = wsol.y

TypeError: only integer scalar arrays can be converted to a scalar index

In [135]:
# Part b : A \ b
import time
import matplotlib.pyplot as plt
from scipy.sparse import spdiags

nu = 0.001
Lx = 20
Ly = 20
nx = 64
ny = 64
N = nx * ny
t = np.arange(0, 4.5, 0.5)  # time array

def ab_rhs(A, w):
    w2 = w.reshape((nx,ny))
    np.linalg.solve(A, w2)
    rhs = nu * np.dot(A, w.flatten()) - (wx * psi_y) + (psi_x * wy).reshape((nx * ny))
    return rhs
    
wsol = solve_ivp(pde, [t[0], t[-1]], w2, t_eval=t , args = (nx, ny, A, B, C, nu, K) , method = 'RK45')
A2 = wsol.y 

start_time = time.time() # Record the start time
end_time = time.time() # Record the end time
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")




TypeError: only integer scalar arrays can be converted to a scalar index